In [44]:
! python aaaaaa.py -c configs/ljs_base.json -m ljs_base

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	RESUME(arg=0, lineno=1039)
           4	LOAD_FAST(arg=0, lineno=1042)
           6	LOAD_CONST(arg=1, lineno=1042)
           8	BINARY_SUBSCR(arg=None, lineno=1042)
          18	LOAD_FAST(arg=0, lineno=1042)
          20	LOAD_CONST(arg=2, lineno=1042)
          22	BINARY_SUBSCR(arg=None, lineno=1042)
          32	COMPARE_OP(arg=4, lineno=1042)
          38	LOAD_FAST(arg=0, lineno=1042)
          40	LOAD_CONST(arg=1, lineno=1042)
          42	BINARY_SUBSCR(arg=None, lineno=1042)
          52	LOAD_FAST(arg=0, lineno=1042)
          54	LOAD_CONST(arg=3, lineno=1042)
          56	BINARY_SUBSCR(arg=None, lineno=1042)
          66	COMPARE_OP(arg=5, lineno=1042)
          72	BINARY_OP(arg=1, lineno=1042)
          76	RETURN_VALUE(arg=None, lineno=1042)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:state.pc

C:\Users\ken\jikken\Jikken5\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")

Batches: 100%|##########| 1/1 [00:00<00:00,  3.28it/s]


In [35]:
!pip install soundfile
